In [1]:
import django
from django.conf import settings
import pathlib
import os
import dotenv
import pandas

In [2]:
BASE_DIR = pathlib.Path('.')

In [3]:
dotenv.load_dotenv(BASE_DIR / '.env')

True

In [4]:
try:
    settings.configure(**{
        'DEBUG': True,
        'BASE_DIR': BASE_DIR,
        'INSTALLED_APPS': [
            'songs'
        ],
        'DATABASES': {
            'default': {
                'ENGINE': 'django.db.backends.sqlite3',
                'NAME': BASE_DIR / 'db.sqlite3',
            }
        },
        'MEDIA_PATH': BASE_DIR / 'media'
    })
except:
    pass

In [5]:
os.environ.setdefault('DJANGO_ALLOW_ASYNC_UNSAFE', 'true')

'true'

In [6]:
try:
    django.setup()
except:
    pass

In [36]:
artists = pandas.read_csv(BASE_DIR / 'media' / 'base_artists_v1.csv', parse_dates=True)

In [37]:
songs = pandas.read_csv(BASE_DIR / 'media' / 'base_songs_v1.csv')

In [38]:
from songs.models import Artist

In [39]:
created_artists = []

In [44]:
def create_artists():
    for item in artists.itertuples(name='Artist'):
        defaults = {
            'name': item.name, 
            'birthname': item.birthname, 
            'spotify_id': item.spotify_id, 
            'genre': item.genre, 
            'spotify_avatar': item.spotify_avatar, 
            'wikipedia_page': item.wikipedia_page
        }

        if not pandas.isna(item.date_of_birth):
            defaults['date_of_birth'] = item.date_of_birth
        
        try:
            artist, _ = Artist.objects.get_or_create(defaults=defaults, name=item.name)
        except Exception as e:
            print(e.args, item)
        else:
            created_artists.append(artist)

In [49]:
def create_songs():
    for item in songs.itertuples(name='Artist'):
        try:
            artist = Artist.objects.get(name=item.artist)
        except:
            print(f'Failed to get: {item}')
        else:
            artist.song_set.get_or_create(
                defaults={
                    'genre': item.genre,
                    'featured_artists': item.featured_artists, 
                    'youtube_id': item.youtube_id,
                    'year': item.year,
                    'difficulty': item.difficulty
                },
                name=item.name
            )

In [ ]:
create_artists()

In [51]:
create_songs()